# Accessing 10x RNA-seq gene expression data

This notebook provides examples and functions for accessing the 10X expression matrix data stored in the ABC Atlas. These files require a large amount of memory to be available to load and analyize them if care is not taken. In this notebook, we present an example of how to access specific gene expressions from the data. The functions used below could be simplily parallized when processing data at scale, however, we leave them simple here.

Care should still be taken not to attempt too load to many genes from the expression matrices.

In [1]:
from typing import List
import os
import pandas as pd
from pathlib import Path
import numpy as np
import anndata
import time

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache
from abc_atlas_access.abc_atlas_cache.anndata_utils import get_gene_data

We will interact with the data using the **AbcProjectCache**. This cache object tracks which data has been downloaded and serves the path to the requsted data on disk. For metadata, the cache can also directly serve up a Pandas Dataframe. See the ``getting_started`` notebook for more details on using the cache including installing it if it has not already been.

**Change the download_base variable to where you have downloaded the data in your system.**

In [2]:
download_base = Path('/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/dat/raw/abc_atlas')
abc_cache = AbcProjectCache.from_s3_cache(download_base)
abc_cache.current_manifest

'releases/20240330/manifest.json'

## Gene expression matrices

The Whole Mouse Brain (WMB) and Whole Human Brain (WHB) datasets are formatted similarly. Each package is formatted as annadata h5ad files with minimal metadata. For each dataset, there are two h5ad files: one storing the raw counts and the other log2 normalization of the counts.

To load the data by gene for either mouse or human dataset, we need to load two pieces of metadata, the ``cell``s table and the ``gene``s table in addition to our instantiated AbcProjectCache object. These metadata can be found in the directories WMB-10X and WHB-10Xv3 for mouse and human respectively. Below we use the human brain data in our example.

In [3]:
ligands = [['Npy'],[ 'Sst'], ['Vip'], ['Tac2'], ['Cck'], ['Penk'], ['Crh'], ['Cort'], ['Tac1'], ['Pdyn'], ['Pthlh'], ['Pnoc'], ['Trh'], ['Grp'], ['Rln1'], ['Adcyap1'], ['Nts'], ['Nmb']]
receptors = [["Npy1r", "Npy2r", "Npy5"], ["Sstr1", "Sstr2", "Sstr3", "Sstr4" ], ["Vipr1", "Vipr2"], ["Tacr3"], ["Cckbr"], ["Oprd1", "Oprm1"], ["Crhr1", "Crhr2"], ["Sstr1", "Sstr2", "Sstr3", "Sstr4"], ["Tacr1"], ["Oprd1", "Oprk1", "Oprm1"], ["Pth1r"], ["Oprl1"], ["Trhr", "Trhr2"
], ["Grpr"], ["Rxfp1", "Rxfp2", "Rxfp3"], ["Adcyap1r1", "Vipr1", "Vipr2"], ["Ntsr1", "Ntsr2"], ["Nmbr"]]

lr_genes = ligands + receptors
lr_genes = [item for sublist in lr_genes for item in sublist]
lr_genes = list(set(lr_genes))
print(len(lr_genes))


47


In [4]:
MERFISH_genes = abc_cache.get_metadata_dataframe(directory='MERFISH-C57BL6J-638850', file_name='gene')
MERFISH_genes = MERFISH_genes[~MERFISH_genes['gene_symbol'].str.startswith('Blank')]
#display(MERFISH_genes.head())

sc_genes = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='gene').set_index('gene_identifier')
display(sc_genes.head())

,gene_symbol,name,mapped_ncbi_identifier,comment
gene_identifier,,,,
ENSMUSG00000051951,Xkr4,X-linked Kx blood group related 4,NCBIGene:497097,NaN
ENSMUSG00000089699,Gm1992,predicted gene 1992,NaN,NaN
ENSMUSG00000102331,Gm19938,"predicted gene, 19938",NaN,NaN
ENSMUSG00000102343,Gm37381,"predicted gene, 37381",NaN,NaN
ENSMUSG00000025900,Rp1,retinitis pigmentosa 1 (human),NCBIGene:19888,NaN


In [5]:
# Keep only genes in SC_genes that are in MERFISH_genes["gene_symbol"] or in lr_genes. Use the gene_symbol as the key
MERFISH_genes_list = MERFISH_genes['gene_symbol'].tolist()
mlr_genes = list(set(MERFISH_genes_list + lr_genes))

target_genes = sc_genes[sc_genes['gene_symbol'].isin(mlr_genes)]
print(target_genes.shape)
display(target_genes.head())

(528, 4)


,gene_symbol,name,mapped_ncbi_identifier,comment
gene_identifier,,,,
ENSMUSG00000025905,Oprk1,"opioid receptor, kappa 1",NCBIGene:18387,NaN
ENSMUSG00000033774,Npbwr1,neuropeptides B/W receptor 1,NCBIGene:226304,NaN
ENSMUSG00000033740,St18,suppression of tumorigenicity 18,NCBIGene:240690,NaN
ENSMUSG00000042501,Cpa6,carboxypeptidase A6,NCBIGene:329093,NaN
ENSMUSG00000042596,Tfap2d,"transcription factor AP-2, delta",NCBIGene:226896,NaN


In [6]:
# metadata for cells for which (library_method is 10Xv3) AND (region_of_interest_acronym is either VIS or MOp)
sc_cells = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='cell_metadata').set_index('cell_label')

sc_cells = sc_cells[(sc_cells["library_method"] == "10Xv3") & (sc_cells["region_of_interest_acronym"].isin(["VIS", "MOp"]))]
sc_cells[["region_of_interest_acronym","library_method"]].value_counts()

print(sc_cells.shape)
display(sc_cells.head(3))

(155383, 15)


,cell_barcode,barcoded_cell_sample_label,library_label,feature_matrix_label,entity,brain_section_label,library_method,region_of_interest_acronym,donor_label,donor_genotype,donor_sex,dataset_label,x,y,cluster_alias
cell_label,,,,,,,,,,,,,,,
CATGGTATCGCATTAG-477_A01,CATGGTATCGCATTAG,477_A01,L8TX_210107_01_C08,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,VIS,Snap25-IRES2-Cre;Ai14-557707,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv3,11.900758,-3.864682,102
CACAGGCGTTCTGACA-480_A04,CACAGGCGTTCTGACA,480_A04,L8TX_210107_01_F09,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,VIS,Snap25-IRES2-Cre;Ai14-556846,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv3,11.421314,-3.986752,102
GTGGAAGCAGCATCTA-480_A04,GTGGAAGCAGCATCTA,480_A04,L8TX_210107_01_F09,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,VIS,Snap25-IRES2-Cre;Ai14-556846,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv3,11.917944,-3.799100,102


### Compiling MOp and VIS scRNAseq data, including the annotations.

In [7]:
import anndata as ad
import toml

paths = toml.load('/allen/programs/celltypes/workgroups/mousecelltypes/Rohan/code/abcex/config.toml')
expr_data = ad.read_h5ad(paths['data']+'expression_matrices/WMB-10Xv3/20230630/WMB-10Xv3-Isocortex-1-log2.h5ad', backed='r')


In [8]:
# all metadata
sc_cells = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='cell_metadata').set_index('cell_label')
sc_cells = sc_cells[(sc_cells["library_method"] == "10Xv3") & (sc_cells["region_of_interest_acronym"].isin(["VIS", "MOp"]))]
sc_cells[["region_of_interest_acronym","library_method"]].value_counts()

region_of_interest_acronym  library_method
MOp                         10Xv3             124501
VIS                         10Xv3              30882
Name: count, dtype: int64

In [9]:
overlapping_cells = np.intersect1d(expr_data.obs.index.values, sc_cells.index.values)
print("expression data: ", np.size(expr_data.obs.index.values))
print("filtered metadata: ", np.size(sc_cells.index.values))
print("filtered metadata cells that have expression data: ", np.size(overlapping_cells))

expression data:  228229
filtered metadata:  155383
filtered metadata cells that have expression data:  90714


In [10]:
# get expr.obs where the index values match sc_cells.index
# adata = expr_data.to_memory()

adata = expr_data[expr_data.obs.loc[overlapping_cells].index, target_genes.index].to_memory()
print(adata.shape)

# annotations for type/region etc. are missing from this file. 
display(adata.obs.head(3))

(90714, 528)


,cell_barcode,library_label,anatomical_division_label
cell_label,,,
AAACCCAAGAACAAGG-480_A04,AAACCCAAGAACAAGG,L8TX_210107_01_F09,Isocortex
AAACCCAAGAACTTCC-198.1_A01,AAACCCAAGAACTTCC,L8TX_200106_01_G07,Isocortex
AAACCCAAGAAGTATC-116_C01,AAACCCAAGAAGTATC,L8TX_190430_01_C08,Isocortex


In [11]:
# get the cluster annotations
cluster_details = abc_cache.get_metadata_dataframe(
    directory='WMB-taxonomy',
    file_name='cluster_to_cluster_annotation_membership_pivoted',
    keep_default_na=False
)
cluster_details.set_index('cluster_alias', inplace=True)
display(cluster_details.head(3))

# get the cluster colors
cluster_colors = abc_cache.get_metadata_dataframe(
    directory='WMB-taxonomy',
    file_name='cluster_to_cluster_annotation_membership_color'
)
cluster_colors.set_index('cluster_alias', inplace=True)
display(cluster_colors.head(3))

# get the roi annotations
roi = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='region_of_interest_metadata')
roi.set_index('acronym', inplace=True)
roi.rename(columns={'order': 'region_of_interest_order',
                    'color_hex_triplet': 'region_of_interest_color'},
           inplace=True)
display(roi.head(3))

,neurotransmitter,class,subclass,supertype,cluster
cluster_alias,,,,,
1,Glut,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0082 L2 IT PPP-APr Glut_3,0326 L2 IT PPP-APr Glut_3
2,Glut,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0082 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
3,Glut,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_2,0322 L2 IT PPP-APr Glut_2


,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color
cluster_alias,,,,,
1,#2B93DF,#FA0087,#0F6632,#266DFF,#64661F
2,#2B93DF,#FA0087,#0F6632,#266DFF,#CCA73D
3,#2B93DF,#FA0087,#0F6632,#002BCC,#99000D


,label,name,region_of_interest_order,region_of_interest_color
acronym,,,,
MO-FRP,WMB-MO-FRP,Somatomotor - Frontal pole,0,#3DCC7C
MOp,WMB-MOp,Primary motor area,1,#179968
SS-GU-VISC,WMB-SS-GU-VISC,Somatosensory/gustatory/visceral areas,2,#2E8599


In [12]:
obs_new_ = sc_cells.loc[overlapping_cells]
obs_new = obs_new_.join(cluster_details, on='cluster_alias')
obs_new = obs_new.join(cluster_colors, on='cluster_alias')
obs_new = obs_new.join(roi[['region_of_interest_order', 'region_of_interest_color']], on='region_of_interest_acronym')
display(obs_new.head(3))

,cell_barcode,barcoded_cell_sample_label,library_label,feature_matrix_label,entity,brain_section_label,library_method,region_of_interest_acronym,donor_label,donor_genotype,...,subclass,supertype,cluster,neurotransmitter_color,class_color,subclass_color,supertype_color,cluster_color,region_of_interest_order,region_of_interest_color
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAACAAGG-480_A04,AAACCCAAGAACAAGG,480_A04,L8TX_210107_01_F09,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,VIS,Snap25-IRES2-Cre;Ai14-556846,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,007 L2/3 IT CTX Glut,0030 L2/3 IT CTX Glut_2,0109 L2/3 IT CTX Glut_2,#2B93DF,#FA0087,#E9530F,#CC7A8F,#AA7ACC,7,#0059CC
AAACCCAAGAACTTCC-198.1_A01,AAACCCAAGAACTTCC,198.1_A01,L8TX_200106_01_G07,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,MOp,Snap25-IRES2-Cre;Ai14-500199,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,006 L4/5 IT CTX Glut,0023 L4/5 IT CTX Glut_1,0069 L4/5 IT CTX Glut_1,#2B93DF,#FA0087,#3DCCB7,#3B9900,#997145,1,#179968
AAACCCAAGAAGTATC-116_C01,AAACCCAAGAAGTATC,116_C01,L8TX_190430_01_C08,WMB-10Xv3-Isocortex-1,cell,NaN,10Xv3,MOp,Snap25-IRES2-Cre;Ai14-457911,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,...,007 L2/3 IT CTX Glut,0030 L2/3 IT CTX Glut_2,0109 L2/3 IT CTX Glut_2,#2B93DF,#FA0087,#E9530F,#CC7A8F,#AA7ACC,1,#179968


In [13]:
# checks
assert np.all(obs_new.index == adata.obs.index), "metadata order is same as adata order"

# replace the metadata with additional annotations
adata.obs = obs_new

display(adata.obs["subclass"].value_counts().to_frame())

,count
subclass,
030 L6 CT CTX Glut,23562
006 L4/5 IT CTX Glut,23177
007 L2/3 IT CTX Glut,19084
032 L5 NP CTX Glut,5445
004 L6 IT CTX Glut,4019
053 Sst Gaba,3123
046 Vip Gaba,2643
005 L5 IT CTX Glut,2362
049 Lamp5 Gaba,1907


In [14]:
adata.write_h5ad('/allen/programs/celltypes/workgroups/mousecelltypes/Donghyun/data/Imputation/10Xv3_VIS_MOp_log2.h5ad')